In [4]:
import random

class ShortSnake:
    def __init__(self, grid_size):
        self.grid_size = grid_size
        self.grid = [[0 for _ in range(grid_size)] for _ in range(grid_size)]
        
        # Init du serpent
        self.snake_pos = [grid_size // 2, grid_size // 2]
        self.snake_dir = "droite"
        self.grid[self.snake_pos[1]][self.snake_pos[0]] = 1

        # Init du fruit
        self.fruit_pos = self.generer_fruit()
        self.grid[self.fruit_pos[1]][self.fruit_pos[0]] = 2

        self.score = 0

    def __str__(self):
        for ligne in self.grid:
            print(" ".join(map(str, ligne)))
        print(" ")
        print("Score : " + self.score.__str__())

    def generer_fruit(self):
        while True:
            x = random.randint(0, self.grid_size - 1)
            y = random.randint(0, self.grid_size - 1)
            if [x, y] != self.snake_pos:
                return [x, y]
            
    def move(self, direction=None):
        # Le serpent prend sa direction initiale, si aucune autre n'est indiquée
        if direction is not None:
            self.snake_dir = direction

        # Calcul de la nouvelle position
        snake_pos_x, snake_pos_y = self.snake_pos
        match(self.snake_dir):
            case "haut":
                snake_pos_y -= 1
            case "bas":
                snake_pos_y += 1
            case "droite":
                snake_pos_x += 1
            case "gauche":
                snake_pos_x -= 1
        
        # Vérification des limite de la grille
        if not (0 <= snake_pos_x < self.grid_size and 0 <= snake_pos_y < self.grid_size):
            print("Collision avec le bord de la grille !")
        if snake_pos_x < 0:
            snake_pos_x = self.grid_size - 1
            self.score -= 10
        elif snake_pos_x >= self.grid_size:
            snake_pos_x = 0
            self.score -= 10
        if snake_pos_y < 0:
            snake_pos_y = self.grid_size - 1
            self.score -= 10
        elif snake_pos_y >= self.grid_size:
            snake_pos_y = 0
            self.score -= 10
        
        # Remise à 0 de la position initial
        self.grid[self.snake_pos[1]][self.snake_pos[0]] = 0
        self.snake_pos = [snake_pos_x, snake_pos_y]
        
        # Génération d'un nouveau fruit si le fruit en cours a été mangé
        if self.snake_pos == self.fruit_pos:
            self.score += 10
            self.fruit_pos = self.generer_fruit()
            self.grid[self.fruit_pos[1]][self.fruit_pos[0]] = 2

        # Mise a jour de la grille si tout est ok
        self.grid[self.snake_pos[1]][self.snake_pos[0]] = 1


if __name__ == "__main__":
    taille = 5
    jeu = ShortSnake(taille)
    jeu.__str__()
    print(' ')
    jeu.move("haut")
    jeu.__str__()
    print(' ')
    jeu.move("haut")
    jeu.__str__()
    print(' ')
    jeu.move("haut")
    jeu.__str__()
    print(' ')
    jeu.move("haut")
    jeu.__str__()

0 0 0 0 0
2 0 0 0 0
0 0 1 0 0
0 0 0 0 0
0 0 0 0 0
 
Score : 0
 
0 0 0 0 0
2 0 1 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
 
Score : 0
 
0 0 1 0 0
2 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
 
Score : 0
 
Collision avec le bord de la grille !
0 0 0 0 0
2 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 1 0 0
 
Score : -10
 
0 0 0 0 0
2 0 0 0 0
0 0 0 0 0
0 0 1 0 0
0 0 0 0 0
 
Score : -10
